# Functional Test 3.1.2 - NVMEs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.2 for testing NVMe attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     ibaldin_0000241998
bastion_key_filename                 /home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  --------------------------------------------------


## Step 3 Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [2]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the test Slice

This creates a VM with two NVMes on a specific worker at a specific site (there are typically at least 2 available at each worker). If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help. 

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [3]:
from datetime import datetime
from dateutil import tz

name='Node1'
nvme_name1='nvme1'
nvme_name2='nvme2'
site='TACC'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.2-NVMe {site} on {worker} on {datetime.now()}"

In [4]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=name, site=site, host=worker, cores=cores, ram=ram, disk=disk)
    
    #Add an NVMe Drive
    node.add_component(model='NVME_P4510', name=nvme_name1)
    #Add another NVMe Drive
    node.add_component(model='NVME_P4510', name=nvme_name2)

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  --------------------------------------------------------------------------------------
Slice Name   Slice Test 3.1.2-NVMe TACC on tacc-w1.fabric-testbed.net on 2022-08-31 01:50:40.103182
Slice ID     20832e92-0c1b-4e04-bf2a-da22be7a88a6
Slice State  StableOK
Lease End    2022-09-01 01:50:43 +0000
-----------  --------------------------------------------------------------------------------------

Retry: 9, Time: 104 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  ---------------  -------  -------
b58f1e42-5f3f-43cd-9fa8-57ffbbf001df  Node1   TACC    tacc-w1.fabric-testbed.net       10     32     100  default_rocky_8  129.114.110.93   Active

Time to stable 104 seconds
Running post_boot_config ... Time to post boot config 105 seconds


## Step 5: Observe the Slice's Attributes

### Print the slice 

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  --------------------------------------------------------------------------------------
Slice Name   Slice Test 3.1.2-NVMe TACC on tacc-w1.fabric-testbed.net on 2022-08-31 01:50:40.103182
Slice ID     20832e92-0c1b-4e04-bf2a-da22be7a88a6
Slice State  StableOK
Lease End    2022-09-01 01:50:43 +0000
-----------  --------------------------------------------------------------------------------------


### Print the node

Each node in the slice has a set of get functions that return the node's attributes. Use the returned `SSH Command` string to check the node. You can do it from a Bash launched inside the Jupyter container.


In [6]:
try:
    node = slice.get_node(name) 
    print(f"{node}")
  
    nvme1 = node.get_component(nvme_name1)
    print(f"{nvme1}")
    
    nvme2 = node.get_component(nvme_name2)
    print(f"{nvme2}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------
ID                 b58f1e42-5f3f-43cd-9fa8-57ffbbf001df
Name               Node1
Cores              10
RAM                32
Disk               100
Image              default_rocky_8
Image Type         qcow2
Host               tacc-w1.fabric-testbed.net
Site               TACC
Management IP      129.114.110.93
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J ibaldin_0000241998@bastion-1.fabric-testbed.net rocky@129.114.110.93
-----------------  ------------------------------------------------------------------------------------------------------------------------
-----------  -------------------------------------
Name         Node1-nvme1
Details      Dell Express Flash NVMe P4510 1TB SFF
Disk (G)     0
Units        1
PCI Address  0000:22:00.0
Model        NVME_P4510
Type         NVME
---

### NVMe PCI Devices

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

View node1's NVMe's

In [7]:
command = "sudo dnf install -q -y pciutils && lspci"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

stdout: 
Installed:
  pciutils-3.7.0-1.el8.x86_64                                                   

00:00.0 Host bridge: Intel Corporation 440FX - 82441FX PMC [Natoma] (rev 02)
00:01.0 ISA bridge: Intel Corporation 82371SB PIIX3 ISA [Natoma/Triton II]
00:01.1 IDE interface: Intel Corporation 82371SB PIIX3 IDE [Natoma/Triton II]
00:01.2 USB controller: Intel Corporation 82371SB PIIX3 USB [Natoma/Triton II] (rev 01)
00:01.3 Bridge: Intel Corporation 82371AB/EB/MB PIIX4 ACPI (rev 03)
00:02.0 VGA compatible controller: Cirrus Logic GD 5446
00:03.0 Ethernet controller: Red Hat, Inc. Virtio network device
00:04.0 SCSI storage controller: Red Hat, Inc. Virtio block device
00:05.0 Unclassified device [00ff]: Red Hat, Inc. Virtio memory balloon
00:06.0 Unclassified device [00ff]: Red Hat, Inc. Virtio RNG
00:07.0 Non-Volatile memory controller: Toshiba Corporation NVMe SSD Controller Cx5 (rev 01)
00:08.0 Non-Volatile memory controller: Toshiba Corporation NVMe SSD Controller Cx5 (rev 01)



## Step 6: Configure NVMe drives

NVMe storage is provided as bare PCI block devices and will need to be partitioned, formated, and mounted before use. Observe the devices.

In [8]:
try:
    nvme1.configure_nvme()
    nvme2.configure_nvme()
except Exception as e:
    print(f"Exception: {e}")

In [9]:
command = "sudo fdisk -l"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

stdout: Disk /dev/vda: 100 GiB, 107374182400 bytes, 209715200 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 512 bytes
I/O size (minimum/optimal): 512 bytes / 512 bytes
Disklabel type: dos
Disk identifier: 0x2cc9c27e

Device     Boot Start       End   Sectors  Size Id Type
/dev/vda1  *     2048 209715166 209713119  100G 83 Linux


Disk /dev/nvme0n1: 894.3 GiB, 960197124096 bytes, 1875385008 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 512 bytes
I/O size (minimum/optimal): 512 bytes / 512 bytes
Disklabel type: gpt
Disk identifier: C386A624-8064-4DEB-A794-BE1B9D30318A

Device         Start        End    Sectors   Size Type
/dev/nvme0n1p1  2048 1875384319 1875382272 894.3G Linux filesystem


Disk /dev/nvme1n1: 894.3 GiB, 960197124096 bytes, 1875385008 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 512 bytes
I/O size (minimum/optimal): 512 bytes / 512 bytes



## Step 7: Cleanup Your Slice

In [10]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")